Importemos las librerías a usar

In [1]:
import pulp as lp
import pandas as pd
import numpy as np
import pickle
import os
import datetime
import statsmodels.api as sm
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, value
import itertools

Llamamos los modelos predictivos a usar.

In [2]:
name_path = 'Input/name.pkl'
cebolla_path = 'Input/cebolla_cabezona.pkl'
platano_path = 'Input/platano_verde.pkl'
tomate_path = 'Input/tomate_chonto.pkl'
ajo_path = 'Input/ajo.pkl'

In [3]:
import glob
import os

# Define the folder path containing the CSV files
folder_path = 'Input/Variables/'

# Use glob to get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Initialize an empty list to store individual DataFrames
dfs = []

# Loop over the list of CSV files and read each into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)
    
    
var_sel_df = pd.concat(dfs, ignore_index=True)

In [4]:
with open(name_path, 'rb') as file:
    name_model = pickle.load(file)

with open(cebolla_path, 'rb') as file:
    cebolla_model = pickle.load(file)
    
with open(platano_path, 'rb') as file:
    platano_model = pickle.load(file)
    
with open(tomate_path, 'rb') as file:
    tomate_model = pickle.load(file)
    
with open(ajo_path, 'rb') as file:
    ajo_model = pickle.load(file)

Vamos a leer las datos de los parametros actuales de los productos.

In [5]:
parametros_base = pd.read_csv('Input\\base_de_datos.csv')

In [6]:
parametros_pivot = pd.pivot_table(parametros_base, values=['precio','Precio medio','unit cost'], index='sku', aggfunc='last').reset_index()

In [7]:
parametros_pivot.loc[parametros_pivot['Precio medio'].isnull(),'Precio medio'] = parametros_pivot.precio
parametros_pivot.loc[parametros_pivot['Precio medio']<parametros_pivot['unit cost'],'Precio medio'] = parametros_pivot.precio
parametros_pivot.loc[parametros_pivot['Precio medio']<parametros_pivot['unit cost'],'Precio medio'] = parametros_pivot['unit cost']

In [8]:
def get_variables(sku, df=var_sel_df):
    df_filtered = df[df.producto==sku]
    return list(df_filtered.sku)

In [9]:
def get_cost(_list,_df=parametros_pivot):
    new_df = _df[_df.sku.isin(_list)==True]
    new_df = new_df.set_index('sku')
    unit_cost_dict = new_df.loc[_list,'unit cost'].to_dict()
    
    return unit_cost_dict

def get_top_price(_list,_df=parametros_pivot):
    new_df = _df[_df.sku.isin(_list)==True]
    new_df = new_df.set_index('sku')
    new_df['Precio medio'] = new_df['Precio medio']*1.2
    unit_cost_dict = new_df.loc[_list,'Precio medio'].to_dict()
    
    return unit_cost_dict

Definimos los sku que predicen nuestros productos top 5.

In [10]:
precios_name = get_variables('BAQ-FRU1-CAT6-64:86:193:194')
precios_cebolla = get_variables('BAQ-FRU1-CAT104105-60271:510131:510132:258690')
precios_platano = get_variables('BAQ-FRU1-CAT1-47:67:151:152')
precios_tomate = get_variables('BAQ-FRU1-CAT104105-305509:1018259:1018260:563293')
precios_ajo = get_variables('BAQ-FRU1-CAT2-346:464:1180:1181')

Definimos los parameotros de restricción de cada producto.

In [11]:
parametros_name = parametros_pivot[parametros_pivot.sku.isin(precios_name)==True]
parametros_cebolla = parametros_pivot[parametros_pivot.sku.isin(precios_cebolla)==True]
parametros_platano = parametros_pivot[parametros_pivot.sku.isin(precios_platano)==True]
parametros_tomate = parametros_pivot[parametros_pivot.sku.isin(precios_tomate)==True]
parametros_ajo = parametros_pivot[parametros_pivot.sku.isin(precios_ajo)==True]

Cargamos los parametros de los modelos.

In [12]:
params_name = name_model.params
params_ajo = ajo_model.params
params_tomate = tomate_model.coef_
params_platano = platano_model.params
params_cebolla = cebolla_model.params

In [13]:
def generate_data(sku,sep):
    precios_sku = get_variables(sku)
    top_val = get_top_price(precios_sku)
    low_val = get_cost(precios_sku)
    
    list_val = list()
    
    for i,j in top_val.items():
        for m,n in low_val.items():
            if i == m:
                list_val.append(list(range(round(n),round(j),sep)))
                
    combined_list = [np.array([combination]) for combination in itertools.product(*list_val)]
    
    return combined_list

In [14]:
def optimization(sku,sep,model,t=0):
    entry_np = generate_data(sku,sep)
    cost_i = list(get_cost(get_variables(sku)).values())[0]
    df = pd.DataFrame()
    cantidad = []
    contribucion = []
    
    
    if t == 0:
        for i in entry_np:
            x = np.hstack([np.ones((i.shape[0], 1)), i])
            q = model.predict(x)
            cantidad.append(q[0])
            contribucion.append(q[0]*(i[0][0]-cost_i))
    elif t == 1:
        for i in entry_np:
            q = model.predict(i)
            cantidad.append(q[0])
            contribucion.append(q[0]*(i[0][0]-cost_i))
    elif t == 2:
        for i in entry_np:
            q = model.predict(i)
            q_i = np.exp(q[0])
            cantidad.append(q_i)
            contribucion.append(q_i*(i[0][0]-cost_i))
    
    df['parametros'] = entry_np
    df['cantidad'] = cantidad
    df['contribucion'] = contribucion
    
    return df[df.contribucion==max(df.contribucion)]

In [15]:
data_platano = generate_data('BAQ-FRU1-CAT1-47:67:151:152',500)

In [16]:
opt_platano = optimization('BAQ-FRU1-CAT1-47:67:151:152',500,platano_model)

In [17]:
data_tomate = generate_data('BAQ-FRU1-CAT104105-305509:1018259:1018260:563293',50)

In [18]:
opt_tomate = optimization('BAQ-FRU1-CAT104105-305509:1018259:1018260:563293',50,tomate_model,1)

In [19]:
data_ajo = generate_data('BAQ-FRU1-CAT2-346:464:1180:1181',50)

In [20]:
opt_ajo = optimization('BAQ-FRU1-CAT2-346:464:1180:1181',50,ajo_model,2)

In [21]:
data_cebolla = generate_data('BAQ-FRU1-CAT104105-60271:510131:510132:258690',50)

In [22]:
opt_cebolla = optimization('BAQ-FRU1-CAT104105-60271:510131:510132:258690',50,cebolla_model)

In [23]:
data_name = generate_data('BAQ-FRU1-CAT6-64:86:193:194',200)

In [24]:
opt_name = optimization('BAQ-FRU1-CAT6-64:86:193:194',200,name_model)

In [25]:
opt_platano['sku'] = 'BAQ-FRU1-CAT1-47:67:151:152'
opt_tomate['sku'] = 'BAQ-FRU1-CAT104105-305509:1018259:1018260:563293'
opt_ajo['sku'] = 'BAQ-FRU1-CAT2-346:464:1180:1181'
opt_cebolla['sku'] = 'BAQ-FRU1-CAT104105-60271:510131:510132:258690'
opt_name['sku'] = 'BAQ-FRU1-CAT6-64:86:193:194'

In [26]:
opt_table = pd.concat([opt_ajo,opt_cebolla,opt_name,opt_platano,opt_name])

In [27]:
opt_table

,parametros,cantidad,contribucion,sku
546119,"[[77950, 6200, 5000]]",46.049794,8.265938e+05,BAQ-FRU1-CAT2-346:464:1180:1181
2033493,"[[4965, 3300, 1100, 2900]]",1935.766512,3.194015e+06,BAQ-FRU1-CAT104105-60271:510131:510132:258690
21908200,"[[5500, 5300, 2400, 2500, 1100, 5800, 4100]]",565.423506,1.583186e+06,BAQ-FRU1-CAT6-64:86:193:194
9983,"[[2600, 8700, 6000, 4300, 2850, 1372, 7500, 24...",1193.294657,5.966473e+05,BAQ-FRU1-CAT1-47:67:151:152
21908200,"[[5500, 5300, 2400, 2500, 1100, 5800, 4100]]",565.423506,1.583186e+06,BAQ-FRU1-CAT6-64:86:193:194


In [49]:
# Define the model
#model = ConcreteModel()

# Define the variables
#model.X = Var(I, domain=NonNegativeReals, bounds=lambda model, i: (b[i], a[i]))

# Demand function
#def demand(model):
#    return sum(c[i] * model.X[i] for i in I) + d

# Objective function
#model.profit = Objective(expr=-demand(model) * (list(dict(model.X).values())[0] - list(b.values())[0]), sense=1)

# Define the solver
#solver = SolverFactory('ipopt')

# Solve the problem
#solver.solve(model, tee=True)

In [28]:
print("Maximum profit:", -value(model.profit))

Maximum profit: 1614159.0069550672


In [152]:
parametros_base.columns

Index(['Unnamed: 0', 'Unnamed: 0_x', 'nro_orden', 'fecha', 'producto',
       'cantidad', 'precio', 'descuento', 'customer_id', 'sku', 'product_id_x',
       'product_quantity_x_step_unit', 'product_step_unit', 'product_unit',
       'sku_parent', 'month', 'totalVentasSinDescuento',
       'totalVentasConDescuento', 'unit cost', 'totalContribucionSinDescuento',
       'totalContribucionConDescuento', 'name', 'category', 'region_code',
       'product_category_id', 'mean_shelf_life', 'promised_lead_time',
       'purchasing_unit', 'buy_unit', 'weight_parameter_apricot',
       'frubana_name', 'Sipsa_name', 'Base_Sipsa', 'Beginning_week',
       'Unnamed: 0.1', 'Producto', 'Mercado mayorista', 'Precio mínimo',
       'Precio máximo', 'Precio medio', 'Tendencia', 'Unnamed: 6', 'Archivo',
       'Unnamed: 0_y', 'Periodo', 'Fecha'],
      dtype='object')

In [163]:
data_real = pd.pivot_table(parametros_base, index=['fecha','sku','producto'], values = ['cantidad','totalContribucionSinDescuento'],aggfunc='sum').reset_index()

In [164]:
top_5 = ['BAQ-FRU1-CAT2-346:464:1180:1181',
         'BAQ-FRU1-CAT6-64:86:193:194',
         'BAQ-FRU1-CAT1-47:67:151:152',
         'BAQ-FRU1-CAT104105-305509:1018259:1018260:563293',
         'BAQ-FRU1-CAT104105-60271:510131:510132:258690']

data_real = data_real[data_real.sku.isin(top_5)==True]

In [165]:
#data_real = data_real[data_real.fecha == max(data_real.fecha)]

In [171]:
data_real[data_real.sku=='BAQ-FRU1-CAT104105-60271:510131:510132:258690']

,fecha,sku,producto,cantidad,totalContribucionSinDescuento
40,2022-07-30,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,951,2723166.0
133,2022-08-01,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,831,2402766.0
232,2022-08-02,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,291,902265.0
332,2022-08-03,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,408,1078644.0
433,2022-08-04,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,1248,2515272.0
...,...,...,...,...,...
11988,2023-01-04,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,348,112359.0
12066,2023-01-05,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,351,115299.0
12154,2023-01-06,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,330,-116277.0
12245,2023-01-07,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,309,-110931.0


In [209]:
margen_real = pd.pivot_table(parametros_base, index=['sku', 'producto'], values=['totalVentasSinDescuento', 'totalContribucionSinDescuento'], aggfunc='sum').reset_index()

In [210]:
margen_real['Margen'] = margen_real.totalContribucionSinDescuento/margen_real.totalVentasSinDescuento

In [211]:
margen_real[margen_real.sku.isin(top_5)==True]

,sku,producto,totalContribucionSinDescuento,totalVentasSinDescuento,Margen
35,BAQ-FRU1-CAT1-47:67:151:152,Plátano Hartón Verde Estandar Desde 2Kg,3.686342e+07,2.370674e+08,0.155498
36,BAQ-FRU1-CAT1-47:67:151:152,Plátano Verde Estándar Mediano Estandar Verde,1.326539e+07,7.046714e+07,0.188249
49,BAQ-FRU1-CAT104105-305509:1018259:1018260:563293,Tomate Chonto Maduración Mixta Estándar (Grand...,6.128463e+07,2.400035e+08,0.255349
58,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,4.476926e+07,1.362526e+08,0.328575
97,BAQ-FRU1-CAT2-346:464:1180:1181,Ajo Estandar Caja,2.181340e+07,4.760010e+08,0.045826
138,BAQ-FRU1-CAT6-64:86:193:194,Ñame Estándar Kg,9.658412e+07,4.412690e+08,0.218878


In [226]:
((sum(parametros_base.precio)/len(parametros_base))-(sum(parametros_base['unit cost'])/len(parametros_base)))/(sum(parametros_base.precio)/len(parametros_base))

0.23885720076027864

In [225]:
b_tomate

{'BAQ-FRU1-CAT104105-305509:1018259:1018260:563293': 2450.0,
 'BAQ-FRU1-CAT6-234:304:750:770': 1372.0,
 'BAQ-FRU1-CAT1-241:367:958:959': 3000.0}

In [229]:
pd.pivot_table(data_real, index= ['sku','producto'],values=['cantidad','totalContribucionSinDescuento'], aggfunc='mean' ).reset_index()

,sku,producto,cantidad,totalContribucionSinDescuento
0,BAQ-FRU1-CAT1-47:67:151:152,Plátano Hartón Verde Estandar Desde 2Kg,231.205645,148642.829073
1,BAQ-FRU1-CAT1-47:67:151:152,Plátano Verde Estándar Mediano Estandar Verde,314.535211,186836.450704
2,BAQ-FRU1-CAT104105-305509:1018259:1018260:563293,Tomate Chonto Maduración Mixta Estándar (Grand...,310.550459,281122.177370
3,BAQ-FRU1-CAT104105-60271:510131:510132:258690,Cebolla Cabezona Blanca Sin Pelar Mixta Kg,514.901408,630552.887324
4,BAQ-FRU1-CAT2-346:464:1180:1181,Ajo Estandar Caja,24.478873,76807.759190
5,BAQ-FRU1-CAT6-64:86:193:194,Ñame Estándar Kg,459.996466,341286.636042


In [244]:
parametros_base.producto[parametros_base.sku.isin(precios_platano)==True].unique()

array(['Papa Blanca Sucia Tamaño Mixto Kg',
       'Plátano Verde Estándar Mediano Estandar Verde',
       'Banano Criollo Estandar Kg', 'Zanahoria Mixta Kg',
       'Plátano Hartón Verde Estandar Desde 2Kg',
       'Cebollín Chino Estándar Atado', 'Durazno Estándar Kg',
       'Lechuga Batavia tamaño mixto Mixta Unidad', 'Lulo Mixto Mixto Kg'],
      dtype=object)